We welcome you all to the Project - 1.zip View in a new window case-based of this course. This project has 2 case studies. The first case study (described below - 30 points) covers concepts taught in Part 1 (first 8 hours of Neural networks basics).

 1st case study - Project 1:

 The case study is from a dataset from Kaggle. 

Link to the Kaggle project site:

https://www.kaggle.com/c/plant-seedlings-classification (Links to an external site.)Links to an external site.

 The dataset has to be downloaded from the above Kaggle web site.

 Can you differentiate a weed from a crop seedling?

The ability to do so effectively can mean better crop yields and better stewardship of the environment.

The Aarhus University Signal Processing group, in collaboration with University of Southern Denmark, has recently released a dataset containing images of approximately 960 unique plants belonging to 12 species at several growth stages.

 

The points distribution for this case is as follows:

1. Read the images and generate the train and test dataset (5 points)
2. Divide the data set into Train and validation data sets
3. Initialize & build the model (10 points)
4. Optimize the model (8 points)
5. Predict the accuracy for both train and validation data (7 points)

## 1. Read the images and generate the train and test dataset (5 points)

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import keras
from google.colab import drive

Using TensorFlow backend.


Connecting/mounting my Google Drive to colab notebook

In [3]:
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Downloading Train and Test Dataset from Keggle to Google Drive

In [0]:
# Colab library to upload files to notebook
from google.colab import files

# Install Kaggle library
!pip install -q kaggle

Copying the files from Kaggle to Google MyDrive

In [0]:
# Upload kaggle API key file
uploaded = files.upload()

Saving kaggle.json to kaggle (1).json


In [0]:
!kaggle competitions download -c plant-seedlings-classification

  0% 0.00/5.13k [00:00<?, ?B/s]
100% 5.13k/5.13k [00:00<00:00, 808kB/s]
test.zip: Skipping, found more recently modified local copy (use --force to force download)
100% 1.60G/1.60G [00:46<00:00, 24.4MB/s]
100% 1.60G/1.60G [00:46<00:00, 36.7MB/s]


Reading/storing dataset for the model building

In [0]:
os.chdir('/content/drive/My Drive/AIML/Res7')

In [0]:
from zipfile import ZipFile
with ZipFile('train.zip', 'r') as z:
  z.extractall()

## 2. Dividing the data set into Train and validation data sets¶


In [0]:
project_path = '/content/drive/My Drive/AIML/Res7/'

In [0]:
train_path = project_path + 'train.zip'
test_path = project_path + 'test.zip'

In [0]:
os.chdir('/content/drive/My Drive/AIML/Res7/')

In [0]:
os.chdir(project_path + 'train/')

In [8]:
os.listdir()

['Fat Hen',
 'Small-flowered Cranesbill',
 'Cleavers',
 'Black-grass',
 'Sugar beet',
 'Shepherds Purse',
 'Charlock',
 'Loose Silky-bent',
 'Scentless Mayweed',
 'Maize',
 'Common Chickweed',
 'Common wheat']

In [9]:
for i in os.listdir() :
  print(i)

Fat Hen
Small-flowered Cranesbill
Cleavers
Black-grass
Sugar beet
Shepherds Purse
Charlock
Loose Silky-bent
Scentless Mayweed
Maize
Common Chickweed
Common wheat


Unziping the files and storing in seperate Train and Test folder in My drive

In [17]:
x_train = []
y_train = []
import cv2
for i in os.listdir():
    print(i)
    if (os.path.isdir(i)):
            for j in os.listdir(i):
                try:
                    dummy = cv2.imread('/content/drive/My Drive/AIML/Res7/train/' + i + "/" + j)
                    dummy = cv2.resize(dummy,(128,128))
                    x_train.append(dummy)
                    y_train.append(i)
                except Exception as e:
                    print(e)

Fat Hen
Small-flowered Cranesbill
Cleavers
Black-grass
Sugar beet
Shepherds Purse
Charlock
Loose Silky-bent
Scentless Mayweed
Maize
Common Chickweed
Common wheat


## Optimize the model (5 points)¶


#### Adaptive Histogram equalization and Contrast stretching¶


In [0]:
## adaptive histogram equalization (AHE) : 

def AHE(img):
    img_adapteq = exposure.equalize_adapthist(x_train, clip_limit=0.03)
    return img_adapteq


In [0]:

# Histogram equalization
 def HE(img):
    img_eq = exposure.equalize_hist(x_train)
    return img_eq

In [0]:

# Contrast stretching
def contrast_stretching(img):
    p2, p98 = np.percentile(img, (2, 98))
    img_rescale = exposure.rescale_intensity(x_train, in_range=(p2, p98))
    return img_rescale


### ImageAugmentation

In [0]:
from keras.preprocessing.image import ImageDataGenerator

# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=50,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=0.5,  # randomly flip images
    vertical_flip=0.5, # randomly flip images
    fill_mode='nearest')
    ##preprocessing_function=contrast_stretching) 
    ##preprocessing_function=AHE)  
    
# Prepare the generator
datagen.fit(x_train)

## 3. Initialize & build the model (10 points)¶


In [0]:
from keras.models import Sequential
from keras.layers import Convolution2D, Dropout, Dense
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.optimizers import adam
# import noise layer
from keras.layers import GaussianNoise

# define noise layer
##layer = GaussianNoise(0.1)

In [15]:
model = Sequential()
model.add(BatchNormalization(input_shape = (128,128,3)))
model.add(Convolution2D(32, (3,3), activation ='relu', input_shape = (128, 128, 3))) 
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.8))
##model.add(GaussianNoise(0.1))

model.add(Convolution2D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.8))
##model.add(GaussianNoise(0.1))

model.add(Convolution2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.8))
## model.add(GaussianNoise(0.1))

model.add(Convolution2D(filters=128, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.8))
##model.add(GaussianNoise(0.1))

model.add(Flatten()) 

# fully connected layer
model.add(Dense(units=128,activation = 'relu'))
model.add(Dense(units = 64, activation = 'relu'))
#model.add(Dropout(0.3))
##model.add(GaussianNoise(0.1))
model.add(Dense(units = 32, activation = 'relu'))
#model.add(Dropout(0.3))
##model.add(GaussianNoise(0.1))
model.add(Dense(units = 12, activation = 'softmax'))

In [19]:
x_train[1].shape


(128, 128, 3)

In [20]:
len(x_train)


4750

In [0]:
x_train = np.array(x_train)


In [0]:
from sklearn.preprocessing import LabelEncoder
encoder =  LabelEncoder()
y_train = encoder.fit_transform(y_train)

In [23]:
y_train


array([5, 5, 5, ..., 4, 4, 4])

In [24]:
len(y_train)


4750

In [0]:
trainY = tf.keras.utils.to_categorical(y_train, num_classes=12)


In [26]:
trainY

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [27]:
from sklearn.model_selection import train_test_split
x_train2, x_val, y_train2, y_val = train_test_split(x_train, trainY, test_size=0.2, random_state=2)
print (len(x_train2))
print (len(x_val))

3800
950


In [28]:
x_train2[0].shape


(128, 128, 3)

In [0]:
x_train2 = x_train2.reshape(x_train2.shape[0],128,128,3)
x_val = x_val.reshape(x_val.shape[0],128,128,3)

In [30]:
print (x_train2.shape)
print (x_val.shape)

(3800, 128, 128, 3)
(950, 128, 128, 3)


In [31]:
print(y_train2.shape)
print(y_val.shape)

(3800, 12)
(950, 12)


 Optimizations Techniques tried while building the model and respective observations

Added Dropout
 Added Noise
 Added Maxpooling

 Dropout and Noise did not helped us much to increase the Test Accuracy for this Dataset

## 5. Predict the accuracy for both train and validation data (5 points)¶


In [32]:

## optimizer = keras.optimizers.Adadelta(lr=0.001)
optimizer = keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=optimizer, loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [33]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 128, 128, 3)       12        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 63, 63, 64)        32832     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 31, 31, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 128)      

In [35]:
batch_size = 64
nb_epochs = 10
history = model.fit(x_train,trainY,
                    epochs=10, 
                    validation_data=(x_val,y_val),
                    verbose = 1,
                    initial_epoch=0)

Train on 4750 samples, validate on 950 samples
Epoch 1/10
4750/4750 [==============================] - 216s 45ms/step - loss: 0.1736 - acc: 0.9360 - val_loss: 0.0934 - val_acc: 0.9558
Epoch 2/10
4750/4750 [==============================] - 216s 45ms/step - loss: 0.1932 - acc: 0.9286 - val_loss: 0.1456 - val_acc: 0.9442
Epoch 3/10
4750/4750 [==============================] - 216s 45ms/step - loss: 0.0936 - acc: 0.9665 - val_loss: 0.0609 - val_acc: 0.9737
Epoch 4/10
4750/4750 [==============================] - 216s 46ms/step - loss: 0.1547 - acc: 0.9480 - val_loss: 0.0585 - val_acc: 0.9779
Epoch 5/10
4750/4750 [==============================] - 216s 45ms/step - loss: 0.1077 - acc: 0.9615 - val_loss: 0.0549 - val_acc: 0.9811
Epoch 6/10
4750/4750 [==============================] - 216s 46ms/step - loss: 0.0802 - acc: 0.9731 - val_loss: 0.0953 - val_acc: 0.9705
Epoch 7/10
4750/4750 [==============================] - 216s 46ms/step - loss: 0.0833 - acc: 0.9718 - val_loss: 0.0361 - val_acc: 0

With 10 Epoc, the accuracy of Train and Test is 97.58% and 97.58% (which i assume, need more turning in terms are regularization, impage augmentation still.